In [31]:
from utils import read_lines

idx_to_name = ['w', 'x', 'y', 'z']
name_to_idx = {name: idx for idx, name in enumerate(idx_to_name)}

class Inp:
    action = 'inp'
    def __init__(self, op1_idx):
        self.op1_idx = op1_idx

    def eval(self, registers, input_val):
        registers[self.op1_idx] = input_val
    
    def __repr__(self):
        return f'inp {idx_to_name[self.op1_idx]}'

class OpBase:
    action = 'base'
    def __init__(self, op1_idx, op2_idx, op2_value):
        self.op1_idx = op1_idx
        self.op2_idx = op2_idx
        self.op2_value = op2_value
    
    def eval(self, registers):
        raise ValueError('Not implemented')
    
    def __repr__(self):
        return f'{self.action} {idx_to_name[self.op1_idx]} {idx_to_name[self.op2_idx] if self.op2_idx is not None else self.op2_value}'
    
class Add(OpBase):
    action = 'add'
    def __init__(self, op1_idx, op2_idx, op2_value):
        super().__init__(op1_idx, op2_idx, op2_value)
    
    def eval(self, registers):
        if self.op2_idx is not None:
            registers[self.op1_idx] += registers[self.op2_idx]
        else:
            registers[self.op1_idx] += self.op2_value

class Mul(OpBase):
    action = 'mul'
    def __init__(self, op1_idx, op2_idx, op2_value):
        super().__init__(op1_idx, op2_idx, op2_value)
    
    def eval(self, registers):
        if self.op2_idx is not None:
            registers[self.op1_idx] *= registers[self.op2_idx]
        else:
            registers[self.op1_idx] *= self.op2_value


class Div(OpBase):
    action = 'div'
    def __init__(self, op1_idx, op2_idx, op2_value):
        super().__init__(op1_idx, op2_idx, op2_value)
    
    def eval(self, registers):
        if self.op2_idx is not None:
            registers[self.op1_idx] //= registers[self.op2_idx]
        else:
            registers[self.op1_idx] //= self.op2_value

class Mod(OpBase):
    action = 'mod'
    def __init__(self, op1_idx, op2_idx, op2_value):
        super().__init__(op1_idx, op2_idx, op2_value)
    
    def eval(self, registers):
        if self.op2_idx is not None:
            registers[self.op1_idx] %= registers[self.op2_idx]
        else:
            registers[self.op1_idx] %= self.op2_value

class Eql(OpBase):
    action = 'eql'
    def __init__(self, op1_idx, op2_idx, op2_value):
        super().__init__(op1_idx, op2_idx, op2_value)
    
    def eval(self, registers):
        if self.op2_idx is not None:
            registers[self.op1_idx] = 1 if registers[self.op1_idx] == registers[self.op2_idx] else 0
        else:
            registers[self.op1_idx] = 1 if registers[self.op1_idx] == self.op2_value else 0

def parse_instruction(line):
    parts = line.split(' ')
    op1_idx = name_to_idx[parts[1]]
    if parts[0] == 'inp':
        return Inp(op1_idx)
    if parts[2] in name_to_idx:
        op2_idx = name_to_idx[parts[2]]
        op2_value = None
    else:
        op2_idx = None
        op2_value = int(parts[2])
    match parts[0]:
        case 'add':
            return Add(op1_idx, op2_idx, op2_value)
        case 'mul':
            return Mul(op1_idx, op2_idx, op2_value)
        case 'div':
            return Div(op1_idx, op2_idx, op2_value)
        case 'mod':
            return Mod(op1_idx, op2_idx, op2_value)
        case 'eql':
            return Eql(op1_idx, op2_idx, op2_value)
        case x:
            raise ValueError(f'unknown instruction {x}')

def parse_input(input_file):
    return [parse_instruction(line) for line in read_lines(input_file)]

def run_inputs(instructions, inputs):
    registers = [0] * 4
    i = 0
    for ins in instructions:
        if ins.action == 'inp':
            print(registers)
            ins.eval(registers, inputs[i])
            i += 1
        else:
            ins.eval(registers)
    return registers

def parse_and_run(instructions, num):
    inputs = [int(i) for i in str(num)]
    if any(x == 0 for x in inputs):
        return None
    return run_inputs(instructions, inputs)

def part1(input_file):
    instructions = parse_input(input_file)
    for num in range(99_999_999_999_999, 11_111_111_111_110, -1):
        inputs = [int(i) for i in str(num)]
        if any(x == 0 for x in inputs):
            continue
        if run_inputs(instructions, inputs)[-1] == 0:
            return num
    

In [ ]:
part1('inputs/day24.txt')

In [42]:
instructions = parse_input('inputs/day24.txt')
sections = []
cur_section = []
for ins in instructions:
    if ins.action == 'inp':
        if cur_section:
            sections.append(cur_section)
            cur_section = []
        cur_section.append(ins)
    else:
        cur_section.append(ins)
sections.append(cur_section)



In [49]:
def run_section(section, input, z):
    registers = [0] * 3 + [z]
    for ins in section:
        if ins.action == 'inp':
            ins.eval(registers, input)
        else:
            ins.eval(registers)
    return registers

In [46]:
z_to_max_input = {}
for input in range(1, 10):
    registers = run_section(sections[0], input, 0)
    z_to_max_input[registers[-1]] = max(z_to_max_input.get(registers[-1], 0), input)
z_to_max_input

{16: 1, 17: 2, 18: 3, 19: 4, 20: 5, 21: 6, 22: 7, 23: 8, 24: 9}

In [57]:
zs = [set() for _ in range(15)]
zs[0].add(0)
for i in range(14):
    for z in zs[i]:
        for input in range(1, 10):
            registers = run_section(sections[i], input, z)
            zs[i+1].add(registers[-1])
    print(i, len(zs[i+1]))


0 9
1 81
2 729
3 810
4 7290
5 7452
6 67068
7 603612
8 5432508


KeyboardInterrupt: 

In [58]:
sections[13]

[inp w,
 mul x 0,
 add x z,
 mod x 26,
 div z 26,
 add x -10,
 eql x w,
 eql x 0,
 mul y 0,
 add y 25,
 mul y x,
 add y 1,
 mul z y,
 mul y 0,
 add y w,
 add y 1,
 mul y x,
 add z y]

In [64]:
zs = set()
for z in range(10000):
    for input in range(1, 10):
        registers = run_section(sections[13], input, z)
        zs.add(registers[-1])
        if registers[-1] == 0:
            print(z, input)


11 1
12 2
13 3
14 4
15 5
16 6
17 7
18 8
19 9


In [63]:
len(zs)

372

In [66]:
def read_puzzle(filename):
    with open(filename) as f:
        return [row.split() for row in f.read().split("\n")]


def get_relevant_adds(puzzle):
    div1, div26 = [], []
    for i in range(0, len(puzzle), 18):
        if puzzle[i + 4][2] == "1":
            div1.append(int(puzzle[i + 15][2]))
            div26.append(None)
        else:
            div1.append(None)
            div26.append(int(puzzle[i + 5][2]))
    return div1, div26


def get_model_no(div1, div26, part1):
    modelNo = [0] * 14
    stack = []
    startDigit = 9 if part1 else 1
    for i, (a, b) in enumerate(zip(div1, div26)):
        if a:
            stack.append((i, a))
        else:
            ia, a = stack.pop()
            diff = a + b
            if part1:
                modelNo[ia] = min(startDigit, startDigit - diff)
                modelNo[i] = min(startDigit, startDigit + diff)
            else:
                modelNo[ia] = max(startDigit, startDigit - diff)
                modelNo[i] = max(startDigit, startDigit + diff)
    return modelNo


def solve(puzzle, part1=True):
    div1, div26 = get_relevant_adds(puzzle)
    return "".join([str(i) for i in  get_model_no(div1, div26, part1)])

In [68]:
solve(read_puzzle('inputs/day24.txt'))

'49917929934999'

In [69]:
solve(read_puzzle('inputs/day24.txt'), False)

'11911316711816'